## Estrategia de Promedio Movil de 40

###### 1. PM20 debe estar por encima del PM40 en HORA
###### 2. Estar en una tendencia Alcista: el PM de 20 sobre el PM de 40 y el de 40 por encima del PM de 100 y este ultimo por encima del PM de 200, (mandatorio debe de ser 20 sobre 40) El PM de 20 y de 40 deben verse paralelos como lineas de ferrocarril.
###### 3. Necesito una caida debemos estar cerca o tocar el PM de 40 en hora 
Que los precios toquen o se acerquen al PM de 4 
Trazo una linea bajista siquiendo la cai a
Cuando una vela verde Alcista, vela final rompe la linea baji ta
despues de las 11:00am se compra un call for 1, 2 days, buy 10 cont y ponle el l mit
Cuando los precios estan lejos del PM40 es zona cara, y cerca del PM de 40 es zona b rata
La mayoria de las veces el mercado sube 1, 2 dias maximo y despues cae nuevamente

In [20]:
import pandas as pd
import numpy as np
#import plotly.graph_objects as go
import yfinance as yf
import talib as ta
#import matplotlib.pyplot as plt
#import math
import os
from datetime import date, timedelta
from scipy import stats
from scipy.signal import argrelextrema
from bokeh.plotting import figure, show, column
from sklearn.linear_model import LinearRegression
from backtesting import Backtest, Strategy

In [25]:
df_h=pd.read_csv(r'D:\PRUEBAS\dataxh.txt',sep='\t')
tickers = [
    'SPY',
    'META',
    'AAPL'
]

#    'SPY',
#    'META',
#    'AAPL',
#    'AMZN',
#    'NFLX',
#    'MRNA',
#    'TSLA',
#    'TNA',
#    'GLD',
#    'SLV',
#    'USO',
#    'BAC',
#    'CVX',
#    'XOM',
#    'QQQ'

In [26]:
df_h['datetime'] = pd.to_datetime(df_h['datetime'])
df = pd.DataFrame()
for ticker in tickers:
    company = df_h.query("companyName==@ticker").copy()
    company.sort_values(by=['datetime'])
    #company = yf.download(ticker, start = ini2_str, end = today_str, interval='60m')
    #company.columns = [company.columns[0][0], company.columns[1][0], company.columns[2][0], company.columns[3][0], company.columns[4][0]]
    #company.rename(columns={'Datetime':'Gmt time'}, inplace = True)
    #company['datetime'] = pd.to_datetime (company.index)
    #company['companyName'] = ticker
    company['SMA20'] = company['Close'].rolling(20).mean()
    company.dropna(inplace=False)
    company['SMA40'] = company['Close'].rolling(40).mean()
    company.dropna(inplace=False)
    company['SMA100'] = company['Close'].rolling(100).mean()
    company.dropna(inplace=False)
    company['SMA200'] = company['Close'].rolling(200).mean()
    company.dropna(inplace=False)
    
    df = pd.concat([df, company],ignore_index=True)

### Detect Pivots/Fractals

In [27]:
ord=10
#for ticker in tickers:
#print("====", ticker)
#dfpl = df[(df.companyName==ticker)] #df.query("companyName=='AAPL'")
max_idx = argrelextrema(df['Close'].values, np.greater, order=ord)[0]
min_idx = argrelextrema(df['Close'].values, np.less, order=ord)[0]
#print(max_idx)
#print(min_idx)
#df['pivotHigh'] = np.nan
#df['pivotLow'] = np.nan
#df['isPivot'] = np.nan
#df['isPivot'] = np.nan
# Aplicar el cálculo solo a los índices en la lista
df.loc[max_idx, 'pivotHigh'] = df['High']+1e-3
df.loc[min_idx, 'pivotLow'] = df['Low']-(1e-3)
df.loc[max_idx, 'isPivot'] = 1
df.loc[min_idx, 'isPivot'] = 2

In [28]:
#CODIGO DE REVISION CARLOS
#dfpl = df.iloc[500:970]
dfpl = df.iloc[0:200]

p = figure(width=2500, height=500,
        title="RCB",
        background_fill_color="#efefef",
        tooltips=[("Index", "@index"), ("Open", "@Open"), ("High","@High"), ("Low","@Low"), ("Close","@Close")]
        )
p.xaxis.major_label_orientation = 0.8 # radians
p.x_range.range_padding = 0.05
#p.xaxis.axis_line_join = "bevel" # radians
p.xaxis.axis_line_width = 2

p.segment("index", "High", "index","Low",  color="black", line_width=1, source=dfpl)

inc = dfpl.query("Close>Open")
dec = dfpl.query("Open>Close")

p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="red",
    line_color="red",    
    source=dec   
)


p.vbar(    
    x="index",
    width=0.6,
    bottom="Open",
    top="Close",
    fill_color="green",
    line_color="green", 
    source=inc   
)

p.scatter(x="index", y="pivotHigh", marker="circle", size=10,
           line_color="navy", fill_color="blue", alpha=0.5, legend_label="Cambio Tendencia Bajista", source=dfpl[(dfpl.isPivot==1)])

p.scatter(x="index", y="pivotLow", marker="circle", size=10,
           line_color="navy", fill_color="green", alpha=0.5, legend_label="Cambio Tendencia Alcista", source=dfpl[(dfpl.isPivot==2)])

show(p)

### Detectando casos

In [29]:
#punto de cambio a tendencia alcista
def pm01(df):
    #if ((df['pivotLow']>0) & ((df['Low']-df['SMA40'])<=1)):
    if ((df['isPivot']==2) & ((df['Low']-df['SMA40'])<=1)):
        return 1
    else:
        return 0
df['pm40_01'] = df.apply(pm01, axis=1)

In [30]:
#el PM20 sobre el PM40, el PM40 por encima del PM100 y este ultimo por encima del PM200
def pm02(df):
    if ((df['SMA20'] >= df['SMA40']) & (df['SMA40'] >= df['SMA100']) & (df['SMA100'] >= df['SMA200'])):
    #if ((df['SMA20'] >= df['SMA40']) & (df['SMA40'] >= df['SMA100'])):
        return 1
    else:
        return 0
df['ind_SMA'] = df.apply(pm02, axis=1)

def pm03(df):
    if ((df['pm40_01'] >= 1) & (df['ind_SMA'] >= 1)):
        return 1
    else:
        return 0

#numeracion de casos donde se dio el cambio de tendencia
#df['id_posiblepm40'] = df.query("pm40_01==1 and ind_SMA==1").groupby(["companyName"]).cumcount()+1
df['id_posiblepm40'] = df.apply(pm03, axis=1)

In [31]:
#df[df["id_posiblepm40"]>0].shape[0]

### Detect Price Channel

In [32]:
def collect_channel(candle, backevalTrend, trendH):
    #localdf = df[candle-backcandles-window:candle-window]
    localdf = dfpl.loc[candle-backevalTrend:candle] #tomar en cuenta el backcandles
    #localdf['isPivot'] = localdf.apply(lambda x: isPivot(x.name,window), axis=1)
    
    #highs = localdf.High.values
    #idxhighs = localdf.High.index
    #lows = localdf.Low.values
    #idxlows = localdf.Low.index

    highs = localdf.loc[localdf.index.isin(trendH)].High.values
    idxhighs = localdf.loc[localdf.index.isin(trendH)].High.index
    #lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    #idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #highs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.values
    #idxhighs = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 1)].High.index
    #lows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.values
    #idxlows = localdf.loc[localdf.index.isin([candle]) | (localdf['isPivot'] == 2)].Low.index

    #lows = localdf.loc[localdf.index.isin(trendL)].Low.values
    #idxlows = localdf.loc[localdf.index.isin(trendL)].Low.index

    #print("highs:",highs)
    #print("idxhighs:",idxhighs)
    #print("lows:",lows)
    #print("idxlows:",idxlows)
    #print ("tamanio:",localdf.shape[0])
    
    #if len(lows)>=2 and len(highs)>=2:
    if len(highs)>=2:
        #sl_lows, interc_lows, r_value_l, _, _ = stats.linregress(idxlows,lows)
        sl_highs, interc_highs, r_value_h, _, _ = stats.linregress(idxhighs,highs)    
        #return(sl_lows, interc_lows, sl_highs, interc_highs, r_value_l**2, r_value_h**2)
        return(sl_highs, interc_highs, r_value_h**2)
    else:
        return(0,0,0)
    

In [33]:
def backcandle(candle):
    
    trendH = []
    backeval = 0
    val = 0
    
    #dfpl['PM40_PM20'] = dfpl['SMA40']-dfpl['SMA20']

    # Tomar los valores hacia atrás desde el índice actual (sin incluirlo)
    prev_vals = dfpl.loc[:candle - 1, 'ind_SMA']

    # Buscar el último índice donde fue 0 (de atrás hacia adelante)
    for i in reversed(prev_vals.index):
        if dfpl.loc[i, 'ind_SMA'] == 0:
            backeval = i - 4
            break

    val = candle - backeval

    
    
    if val>75:
        backeval=candle-75
        
    if (dfpl.loc[backeval,"ind_SMA"]>0):
        idx_negativo = dfpl.loc[:backeval, 'ind_SMA'][::-1].lt(0).idxmax()
        backeval = idx_negativo - 4

    if val>75:
        backeval=candle-75

    #ini = candle - backeval
    ini = backeval
    trendprev=dfpl.loc[ini:candle]
    trend = trendprev.reset_index(drop=False)

    idxhighs = trend.index.to_numpy().reshape(-1,1) # Convertir X a una matriz 2D
    highs = trend.High.values
    modelo1 = LinearRegression()
    modelo1.fit(idxhighs, highs)
    # Obtener la predicción de la línea de regresión
    Y_pred = modelo1.predict(idxhighs)
    # Calcular la distancia solo de los puntos que están POR ENCIMA de la línea
    trend['Distancia'] = trend["High"] - Y_pred  # Diferencia entre el valor real y la predicción
    # Filtrar los puntos que están por encima (donde Y > Y_pred, es decir, Distancia > 0)
    df_encima = trend[trend['Distancia'] > 0]
    puntos_mas_lejanos = df_encima.nlargest(2, 'Distancia')   #2 puntos mas lejanos
    trendH = puntos_mas_lejanos["index"].tolist()
    trendH.append(candle)   
    
    return trendH, backeval

### Detect Break Out

In [34]:
def trendChannel(row,candle, backeval, window, sl_highs, interc_highs):

    if (candle-backeval-window)<0:
        return np.nan
    ini=candle-backeval
    fin=candle+window
    trendcurrhigh = np.where(np.logical_or((row.index > fin),  (row.index <  ini)), np.nan, (sl_highs*row.index + interc_highs))
    return trendcurrhigh

In [35]:
def isBreakOut(candleEval,backcandles, window, start, candle):
    if (candleEval-backcandles-window)<0:
        return 0

    if candleEval==start:
        prev_idx = candleEval
    else:
        prev_idx = candleEval-1
        
    prev_high = dfpl.loc[prev_idx].High
    prev_low = dfpl.loc[prev_idx].Low
    prev_close = df.loc[prev_idx].Close
    
    curr_idx = candleEval
    curr_high = dfpl.loc[candleEval].High
    curr_low = dfpl.loc[candleEval].Low
    curr_close = dfpl.loc[candleEval].Close
    curr_open = dfpl.loc[candleEval].Open

    #trend_prevlow=dfpl.loc[prev_idx].trendcurrlow
    trend_prevhigh=dfpl.loc[prev_idx].trendcurrhigh

    #trend_currlow=dfpl.loc[candleEval].trendcurrlow
    trend_currhigh=dfpl.loc[candleEval].trendcurrhigh

    #codigo para tendencia LOW, nos importa breaks en tendencia HIGH
    #if ( prev_high > trend_prevlow and
    #    prev_close < trend_prevlow and
    #    curr_open < trend_currlow and
    #    curr_close < trend_currlow and candleEval>candle): #and r_sq_l > 0.9
    #    return 1
    
    if ( prev_low < trend_prevhigh and
        prev_close > trend_prevhigh and
        curr_open > trend_currhigh and
        curr_close > trend_currhigh and
        candleEval>candle): #and r_sq_h > 0.9
        return 2

    elif (curr_open > trend_currhigh and
        curr_close > trend_currhigh and candleEval>candle):
        return 3
    
    else:
        return np.nan

In [36]:
def breakpointpos(x):
    if x['isBreakOutIni'] in (2,3):
        return x['Low']+3e-3
    elif x['isBreakOutIni']==1:
        return x['High']-3e-3
    else:
        return np.nan

### Hallando casos de Ruptura techo

In [37]:
def revisionVelas(dfpl, backeval,window, candle):
    #Revision de Velas
    #Promedio del volumen de 50 velas antes del posible caso
    promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    #puede darse cualquiera de estas condiciones: 
    # - que la vela sea verde, que tenga volumen, este por encima de la linea trend y que tenga cuerpo (cierra por encima de su tercera parte)
    # - que la vela sea envolvente, alcista y que este por encima de la linea trend
    j=candle+1
    k=candle+window
    ind_vela=0
    
    #k no puede pasrse del rango
    maxindex = dfpl.index[-1]
    if (k>maxindex):
        k=maxindex
    
    #print("candle:",candle)
    #print("window:",window)
    #print("j:",j, ",k:",k)
    
    #Evaluacion de velas ALCISTAS
    dfpl["cdlengulfing"] = ta.CDLENGULFING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlhammer"] = ta.CDLHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmorningstar"] = ta.CDLMORNINGSTAR(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlpiercing"] = ta.CDLPIERCING(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlclosingmarubozu"] = ta.CDLCLOSINGMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlmarubozu"] = ta.CDLMARUBOZU(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdl3whitesoldiers"] = ta.CDL3WHITESOLDIERS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharami"] = ta.CDLHARAMI(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlharamicross"] = ta.CDLHARAMICROSS(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlinvertdhammer"] = ta.CDLINVERTEDHAMMER(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])
    dfpl["cdlladderbottom"] = ta.CDLLADDERBOTTOM(dfpl['Open'], dfpl['High'], dfpl['Low'], dfpl['Close'])

    dfpl['Vela_Verde'] = dfpl['Close'] > dfpl['Open']
    # Calcular el nivel de la tercera parte
    #dfpl['Third_Level'] = dfpl['Low'] + (dfpl['High'] - dfpl['Low']) / 3
    dfpl['Third_Level'] = dfpl['Open'] + (dfpl['High'] - dfpl['Low']) / 3
    # Verificar si la vela cierra por encima de su tercera parte, vela con cuerpo
    dfpl['Above_Third'] = dfpl['Close'] > dfpl['Third_Level']

    while j<=k:            
        
        vol = dfpl.loc[j,"Volume"]
        above_Third = dfpl.loc[j,"Above_Third"]
        vela_Verde = dfpl.loc[j,"Vela_Verde"]
        lineTrend = dfpl.loc[j,'trendcurrhigh']
        high = dfpl.loc[j,'High']
        close = dfpl.loc[j,'Close']

        cdlengulfing = dfpl.loc[j,"cdlengulfing"]
        cdlhammer = dfpl.loc[j,"cdlhammer"]
        cdlmorningstar = dfpl.loc[j,"cdlmorningstar"]
        cdlpiercing = dfpl.loc[j,"cdlpiercing"]
        cdlclosingmarubozu = dfpl.loc[j,"cdlclosingmarubozu"]
        cdlmarubozu = dfpl.loc[j,"cdlmarubozu"]
        cdl3whitesoldiers = dfpl.loc[j,"cdl3whitesoldiers"]
        cdlharami = dfpl.loc[j,"cdlharami"]
        cdlharamicross = dfpl.loc[j,"cdlharamicross"]
        cdlinvertdhammer = dfpl.loc[j,"cdlinvertdhammer"]
        cdlladderbottom = dfpl.loc[j,"cdlladderbottom"]

        #vela consecutiva alcista
        candle_up = (#dfpl.loc[j,"Vela_Verde"]==True 
                     #and dfpl.loc[j-1,"Vela_Verde"]==True 
                     dfpl.loc[j,"High"]> dfpl.loc[j-1,"High"]
                     and dfpl.loc[j-1,"High"]> dfpl.loc[j,"Low"]
                     and dfpl.loc[j,"Low"]> dfpl.loc[j-2,"High"]
                     and dfpl.loc[j-2,"High"]> dfpl.loc[j-1,"Low"]
                     and dfpl.loc[j-1,"Low"]> dfpl.loc[j-3,"High"]
                     and dfpl.loc[j-3,"High"]> dfpl.loc[j-2,"Low"]
                     and dfpl.loc[j-2,"Low"]> dfpl.loc[j-3,"Low"]
                     #and dfpl.loc[j,"Close"]> dfpl.loc[j-1,"High"] 
                     #and dfpl.loc[j-1,"Close"]>dfpl.loc[j-2,"High"]
                     #and dfpl.loc[j-1,"Close"]>dfpl.loc[j-3,"High"] 
                    )

        if (candle_up==True and close>lineTrend):
            ind_vela=ind_vela+1
        #vela sea verde, que tenga volumen, que tenga cuerpo y que este por encima de la linea trend
        #if (vela_Verde==True and promVol<=vol and above_Third==True and close>=lineTrend):
        #    ind_vela=ind_vela+1
        
        #vela sea envolvente, alcista y que este por encima de la linea trend
        #if (
        #    (cdlhammer>0 or
        #     cdlmorningstar>0 or
        #     cdlpiercing>0 or
        #     cdl3whitesoldiers>0 or
        #     cdlharami>0 or
        #     cdlharamicross>0 or
        #     cdlinvertdhammer>0 or
        #     cdlladderbottom>0
        #    )
        #    and promVol<=vol
        #    and close>=lineTrend):
        #    ind_vela=ind_vela+1

        #vela sea envolvente, alcista y que este por encima de la linea trend
        #if (
        #    (cdlclosingmarubozu>0 or
        #     cdlmarubozu>0 or
        #     cdlengulfing>0 or
        #     cdlhammer>0
        #    )
        #    #and promVol<=vol
        #    and close>=lineTrend):
        #    ind_vela=ind_vela+1 
            
        j=j+1
    
    return 1 #1 para mostrar todo ind_vela para que funcione el filtro

In [38]:
df_casos = pd.DataFrame()
caso = 0
ticker2 = ""
window=5
backeval=75
backevalTrend=0
dfpl = pd.DataFrame
trendH = []
trendL = []
cant = 0
for i, row in df.iterrows():
    #if (i<=200):
    if(df['id_posiblepm40'][i]==1): #posibles casos 
        #print("posible caso:",i, df['companyName'][i])
        cant = 0
        candle = i
        cnt = 0
        valiniHigh = df.loc[candle,"High"]
        #valiniLow = df.loc[candle,"Low"]
        ticker = df['companyName'][i]
        #Reinicio de casos por company
        if ticker2 != ticker:
            ticker2 = ticker
            caso = 0
        dfpl = df[(df.companyName==ticker)].loc[i-backeval:i+backeval]       

        #print("rowMin:", dfpl.index[0],", rowMax:", dfpl.index[-1])
                    
        h=1
        l=1

        cant = cant +1
        cantHorasTrend=0
        #print("opcionH:",h,"opcionL:",l)
        trendH, backevalTrend = backcandle(candle)
        #print ("-->trendH:",trendH)
        #print ("-->backevalTrend:",backevalTrend)    
        
        cantHorasTrend = (dfpl.loc[candle,"datetime"] - dfpl.loc[backevalTrend,"datetime"]).total_seconds() / 3600

        if (cantHorasTrend>48): #Verificacion de caida por lo menos 2 dias
            backevalTrend2 = candle - backevalTrend
            #print ("backevalTrend2", backevalTrend2)
                        
            #revision de trend negativo indica que esta cayendo, solo me quedo con estos casos
            sl_highs, interc_highs, r_sq_h = collect_channel(candle, backevalTrend2, trendH)
                    
    
            #print("======:", candle)
            #print("===trendH===")
            #print(trendH)
            #print("===trendL===")
            #print(trendL)
            dfpl.loc[trendH, "trendH"] = 1
        
            
            #print("sl_highs:", sl_highs, ", sl_lows:", sl_lows, ", r_sq_h:", r_sq_h)
            if (sl_highs<=0): #solo tendencias a la baja and ind_vela>0 sl_lows<0 and r_sq_h>=0.7  and sl_highs<=-0.1 and ind_vela>0
                
                #crear linea de tendencia high y low
                dfpl['trendcurrhigh']  = trendChannel(dfpl,candle,backevalTrend2,window,sl_highs,interc_highs)
                
                ind_vela = revisionVelas(dfpl, backeval,window, candle)
                promVol = dfpl.loc[i-backeval:i]["Volume"].mean()
    
                #print("ind_vela:",ind_vela)
                if (ind_vela>0):
                    dfpl["bearishSlope"] = np.nan #pendiente bajista en vacio
                    dfpl.loc[i, "bearishSlope"] = 1 #pendiente bajista solo al punto evaluado            
                    dfpl["ind_posicion"] = 0
                    dfpl.loc[(dfpl.index < i), 'ind_posicion'] = -1
                    dfpl.loc[(dfpl.index > i), 'ind_posicion'] = 1
                    dfpl["promVol"] = promVol
                    dfpl["sl_highs"] = sl_highs
                    dfpl["r_sq_h"] = r_sq_h
            
                    start = dfpl.index[0]
                    dfpl["isBreakOut"] = [isBreakOut(candleEval, backevalTrend2, window, start, candle) for candleEval in dfpl.index]
                    # Solo me quedo con el primer BREAK OUT
                    cnt = dfpl.query("isBreakOut in (2,3) and Close>Open").shape[0] 
                    id=0
                    id2=0
                    if cnt>0:
                        id = dfpl.query("isBreakOut in (2,3) and Close>Open").index[0]                        
                        #dfpl['isBreakOutIni'] = np.where(id!=dfpl.index, np.nan, dfpl['isBreakOut'])                        
                        dfpl.loc[id,'isBreakOutIni'] = 1
                        dfpl['breakpointpos'] = dfpl.apply(lambda row: breakpointpos(row), axis=1)
                        cnt2 = dfpl.query("index>@id and isPivot==1").shape[0]
                        
                        if cnt2>0:
                            id2 = dfpl.query("index>@id and isPivot==1").index[0]
                            dfpl.loc[id2,'isBreakOutFinal'] = 1
                        else:
                            dfpl['isBreakOutFinal'] = np.nan                      
                        
                    else:
                        dfpl['isBreakOutIni'] = np.nan                        
                        dfpl['breakpointpos'] = np.nan
            
                    if cnt>0:
                        #INSERT CASO
                        caso=caso+1                            
                        dfpl["caso"] = caso
                        print(i, df['companyName'][i])
                        if len(df_casos)<1:
                            df_casos = dfpl
                        else:
                            df_casos = pd.concat([df_casos, dfpl],ignore_index=False)


239 SPY
275 SPY
915 SPY
985 SPY
1101 SPY
1174 SPY
1236 SPY
1271 SPY
1334 SPY
1357 SPY
1387 SPY
1440 SPY
1462 SPY
1726 SPY
2183 SPY
2415 SPY
2684 SPY
2920 SPY
3380 SPY
3789 META
4147 META
4588 META
4664 META
4726 META
4756 META
4824 META
5386 META
5622 META
5885 META
6167 META
6883 META
6915 META
7226 AAPL
7910 AAPL
7945 AAPL
8640 AAPL
8706 AAPL
8737 AAPL
8782 AAPL
8874 AAPL
9398 AAPL
9650 AAPL
9706 AAPL
10410 AAPL


In [39]:
#descarga de informacion, estrategia: RUPTURA DEL CANAL BAJISTA
path = r'D:\PRUEBAS\pm40_h.txt'
# check whether the file exists
if os.path.exists(path):
    # delete the file
    os.remove(path)
else:
    # if the file does not exist.
    print("File does not exists. File needs to be created.")

#export DataFrame to text file
with open(path, 'a') as f:
    #df_string = appl_hor3.to_string(header=True, index=False, sep ='\t')
    df_casos.to_csv(path, header=True, index=None, sep='\t', mode='w')
    #f.write(df_string)

##### BACKTESTING
###### Usando paquete backtesting.py

In [40]:
df_casos2 = df_casos.query("isBreakOutIni==1 or isBreakOutFinal==1").copy()

In [41]:
# =============== BACKTESTING ================= #
# obteniendo dataframe backtesting
df_backTesting = pd.merge(df, df_casos2[['datetime','companyName', 'isBreakOutIni','isBreakOutFinal','caso']], on = ['companyName','datetime'], how='left')
# datos de trade se asigna una vela anterior debido a que backtesting.py toma entra como la siguiente vela
df_backTesting['isBreakOutIni'] = df_backTesting['isBreakOutIni'].shift(-1)
df_backTesting['isBreakOutFinal'] = df_backTesting['isBreakOutFinal'].shift(-1)
#modificamos indice con el compo datetime
df_backTesting.set_index('datetime', inplace=True)

In [42]:
#Strategia
class strategyRupturaCanal(Strategy):    
    def init(self):        
        self.breakout_entry = self.I(lambda x: x, self.data.isBreakOutIni) #Indica la señal entrada al backtesting
        self.breakout_exit = self.I(lambda x: x, self.data.isBreakOutFinal) #Indica la señal salida al backtesting

    def next(self):       
        #print(self.data)        
        if self.breakout_entry[-1] == 1:
            self.buy()  #COMPRA
        elif self.breakout_exit[-1] == 1 and self.position:
            self.position.close() #CIERRE

In [43]:
# Por cada Ticket
my_stats = {}
estadisticas = pd.DataFrame()    # lista para ticker
tradesprev = pd.DataFrame()     # lista para cada caso por ticker
for ticker in df_backTesting['companyName'].unique():
    ticker_data = df_backTesting[df_backTesting['companyName'] == ticker].copy() #Filtro por cada ticker
    #ticker_data=ticker_data.sort_values(by='caso')
    #print("\n" + "="*50 + "\n")
    bt = Backtest(ticker_data, strategyRupturaCanal, cash=10_000)
    #stats = bt.run()
    #my_stats[f"stats_BreakOUT_{ticker}"] = bt.run()
    my_stats = bt.run()
    listTrades = my_stats['_trades']
    tradesdf = pd.DataFrame(listTrades) #trades
    tradesdf['Ticker'] = ticker
    my_stast_filter=pd.DataFrame([my_stats])
    if tradesprev.shape[0]==0:
        tradesprev = tradesdf.copy()
        estadisticas=my_stast_filter.copy()
    else:
        tradesprev = pd.concat([tradesprev, tradesdf],ignore_index=True)
        estadisticas = pd.concat([estadisticas, my_stast_filter],ignore_index=True)
    # Mostramos
    print(f"Resultados para {ticker}:")
    print(stats)   
    #plt.title(f"Backtest -  {ticker}")
    bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")

Backtest.run:   0%|          | 0/3219 [00:00<?, ?bar/s]

Resultados para SPY:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_14376\2904632593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/3176 [00:00<?, ?bar/s]

Resultados para META:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_14376\2904632593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


Backtest.run:   0%|          | 0/3212 [00:00<?, ?bar/s]

C:\Users\LINDER\anaconda3\Lib\site-packages\backtesting\backtesting.py:954: UserWarning: time=1761: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Resultados para AAPL:
<module 'scipy.stats' from 'C:\\Users\\LINDER\\anaconda3\\Lib\\site-packages\\scipy\\stats\\__init__.py'>


C:\Users\LINDER\AppData\Local\Temp\ipykernel_14376\2904632593.py:27: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot(resample=False, filename=f"plot_BreakOUT_{ticker}")


In [44]:
estadisticas

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,8.796562,12533.105072,12533.105072,25.331051,28.891968,12.052506,3.509863,...,1.291523,10 days 00:00:00,2 days 23:00:00,NaN,1.295720,6.196425,NaN,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice Exit...
1,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,5.015764,17706.955994,17726.530334,77.069560,104.702589,33.371959,21.634732,...,4.960565,6 days 23:00:00,2 days 14:00:00,NaN,5.074754,4.016727,NaN,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice Exit...
2,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,7.650430,13428.965549,13955.645065,34.289655,2.293580,16.020157,7.419013,...,3.017855,10 days 00:00:00,5 days 10:00:00,9.478018,3.066061,2.728868,0.785938,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice ExitP...


In [45]:
ini = df_casos2[(df_casos2['isBreakOutIni']==1)].copy()
df_casos3_ini=ini.rename(columns={'Open': 'EntryPrice', 'datetime': 'EntryTime', 'companyName': 'Ticker'})[['EntryPrice','Ticker','caso','EntryTime']]
final = df_casos2[(df_casos2['isBreakOutFinal']==1)].copy()
df_casos3_final=final.rename(columns={'Open': 'ExitPrice', 'datetime': 'ExitTime', 'companyName': 'Ticker'})[['ExitPrice','Ticker','caso','ExitTime']]

In [46]:
df_casos3=df_casos3_ini.merge(df_casos3_final,on=['Ticker','caso'],how='outer')

In [47]:
tradesprev[tradesprev['Ticker']=='SPY'].shape[0]

18

In [48]:
df_casos3[df_casos3["Ticker"]=='SPY'].shape[0]

19

In [49]:
trades = pd.merge(df_casos3, tradesprev, on = ['EntryTime','Ticker'], how='left')

In [50]:
trades[trades["Ticker"]=='SPY'].shape[0]

19

In [51]:
def exitPrice(x):
    if pd.isna(x['ExitPrice_x']):
        return x['ExitPrice_y']
    else:
        return x['ExitPrice_x']
trades['ExitPrice'] = trades.apply(lambda row: exitPrice(row), axis=1)

In [52]:
trades = trades.rename(columns={'EntryPrice_x': 'EntryPrice'})
trades = trades.drop(['EntryPrice_y','ExitPrice_x', 'ExitPrice_y'], axis=1)

In [53]:
def exitTime(x):
    if pd.isna(x['ExitTime_x']):
        return x['ExitTime_y']
    else:
        return x['ExitTime_x']
trades['ExitTime'] = trades.apply(lambda row: exitTime(row), axis=1)

In [54]:
trades[trades['ExitTime_x'].isnull()]

,EntryPrice,Ticker,caso,EntryTime,ExitTime_x,Size,EntryBar,ExitBar,SL,TP,PnL,ReturnPct,ExitTime_y,Duration,Tag,Entry_λ(isBreakOu…),Exit_λ(isBreakOu…),ExitPrice,ExitTime


In [55]:
def duration(x):
    if pd.isna(x['Duration']):
        if pd.isna(x['ExitTime']):
            return pd.NaT
        else:
            return x['ExitTime']-x['EntryTime']
    else:
        return x['Duration']
trades['Duration'] = trades.apply(lambda row: duration(row), axis=1)

In [56]:
trades = trades.drop(['ExitTime_x', 'ExitTime_y'], axis=1)

In [57]:
trades

,EntryPrice,Ticker,caso,EntryTime,Size,EntryBar,ExitBar,SL,TP,PnL,ReturnPct,Duration,Tag,Entry_λ(isBreakOu…),Exit_λ(isBreakOu…),ExitPrice,ExitTime
0,193.410004,AAPL,1,2023-07-24 13:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 days 02:00:00,NaN,NaN,NaN,196.199997,2023-07-28 15:30:00+00:00
1,192.470001,AAPL,2,2023-12-11 18:30:00+00:00,51.0,934.0,951.0,None,None,316.709564,0.032265,2 days 20:00:00,None,NaN,NaN,198.679993,2023-12-14 14:30:00+00:00
2,195.250000,AAPL,3,2023-12-18 17:30:00+00:00,52.0,968.0,982.0,None,None,109.200317,0.010755,2 days 00:00:00,None,NaN,NaN,197.350006,2023-12-20 17:30:00+00:00
3,182.729996,AAPL,4,2024-05-10 18:30:00+00:00,57.0,1663.0,1686.0,None,None,443.802612,0.042609,5 days 19:00:00,None,NaN,NaN,190.516006,2024-05-16 13:30:00+00:00
4,188.820007,AAPL,5,2024-05-24 13:30:00+00:00,57.0,1728.0,1763.0,None,None,237.119339,0.022032,10 days 00:00:00,None,NaN,1.0,192.979996,2024-06-03 13:30:00+00:00
5,190.550003,AAPL,6,2024-05-31 18:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 days 19:00:00,NaN,NaN,NaN,192.979996,2024-06-03 13:30:00+00:00
6,200.892502,AAPL,7,2024-06-11 14:30:00+00:00,55.0,1807.0,1817.0,None,None,881.925201,0.079819,1 days 03:00:00,None,NaN,NaN,216.927505,2024-06-12 17:30:00+00:00
7,215.009995,AAPL,8,2024-07-01 15:30:00+00:00,55.0,1899.0,1941.0,None,None,921.250000,0.077903,9 days 04:00:00,None,NaN,NaN,231.759995,2024-07-10 19:30:00+00:00
8,230.345001,AAPL,9,2024-10-16 16:30:00+00:00,56.0,2421.0,2445.0,None,None,324.239624,0.025136,5 days 03:00:00,None,NaN,NaN,236.134995,2024-10-21 19:30:00+00:00
9,244.479996,AAPL,10,2024-12-09 15:30:00+00:00,54.0,2674.0,2688.0,None,None,267.299835,0.020247,2 days 00:00:00,None,NaN,NaN,249.429993,2024-12-11 15:30:00+00:00


In [58]:
trades=trades[['Ticker','EntryTime','ExitTime','EntryPrice','ExitPrice','Duration','Size','EntryBar','ExitBar','ReturnPct','caso']]

In [59]:
trades.shape[0]

44

In [60]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   Ticker      44 non-null     object             
 1   EntryTime   44 non-null     datetime64[ns, UTC]
 2   ExitTime    44 non-null     datetime64[ns, UTC]
 3   EntryPrice  44 non-null     float64            
 4   ExitPrice   44 non-null     float64            
 5   Duration    44 non-null     timedelta64[ns]    
 6   Size        40 non-null     float64            
 7   EntryBar    40 non-null     float64            
 8   ExitBar     40 non-null     float64            
 9   ReturnPct   40 non-null     float64            
 10  caso        44 non-null     int64              
dtypes: datetime64[ns, UTC](2), float64(6), int64(1), object(1), timedelta64[ns](1)
memory usage: 3.9+ KB


In [61]:
estadisticas

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Avg. Trade [%],Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,_strategy,_equity_curve,_trades
0,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,8.796562,12533.105072,12533.105072,25.331051,28.891968,12.052506,3.509863,...,1.291523,10 days 00:00:00,2 days 23:00:00,NaN,1.295720,6.196425,NaN,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice Exit...
1,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,5.015764,17706.955994,17726.530334,77.069560,104.702589,33.371959,21.634732,...,4.960565,6 days 23:00:00,2 days 14:00:00,NaN,5.074754,4.016727,NaN,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice Exit...
2,2023-05-31 13:30:00+00:00,2025-05-28 10:00:00+00:00,727 days 20:30:00,7.650430,13428.965549,13955.645065,34.289655,2.293580,16.020157,7.419013,...,3.017855,10 days 00:00:00,5 days 10:00:00,9.478018,3.066061,2.728868,0.785938,strategyRupturaCanal,Equity Drawd...,Size EntryBar ExitBar EntryPrice ExitP...


In [63]:
path = r'D:\PRUEBAS'

path_trades = os.path.join(path, 'trades_pm40.txt')
path_estadisticas = os.path.join(path, 'estadisticas_pm40.txt')

for file_path in [path_trades, path_estadisticas]:
    if os.path.exists(file_path):
        os.remove(file_path)
    else:
        print(f"File does not exist: {file_path}. It will be created.")

trades.to_csv(path_trades, header=True, index=False, sep='\t', mode='w')
estadisticas.to_csv(path_estadisticas, header=True, index=False, sep='\t', mode='w')

File does not exist: D:\PRUEBAS\trades_pm40.txt. It will be created.
File does not exist: D:\PRUEBAS\estadisticas_pm40.txt. It will be created.
